<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Creation-of-Database" data-toc-modified-id="Creation-of-Database-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Creation of Database</a></span></li><li><span><a href="#Creations-of-the-tables" data-toc-modified-id="Creations-of-the-tables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creations of the tables</a></span><ul class="toc-item"><li><span><a href="#Table-structure" data-toc-modified-id="Table-structure-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Table structure</a></span></li><li><span><a href="#Table-basics" data-toc-modified-id="Table-basics-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Table basics</a></span></li><li><span><a href="#table-ratings" data-toc-modified-id="table-ratings-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>table ratings</a></span></li><li><span><a href="#Inserting-the-data" data-toc-modified-id="Inserting-the-data-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Inserting the data</a></span></li></ul></li><li><span><a href="#Playing-with-queries" data-toc-modified-id="Playing-with-queries-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Playing with queries</a></span><ul class="toc-item"><li><span><a href="#Function-to-ask-the-queries" data-toc-modified-id="Function-to-ask-the-queries-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Function to ask the queries</a></span></li><li><span><a href="#Simple-queries" data-toc-modified-id="Simple-queries-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Simple queries</a></span><ul class="toc-item"><li><span><a href="#How-many-different-genre-combinations-are-there" data-toc-modified-id="How-many-different-genre-combinations-are-there-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>How many different genre combinations are there</a></span></li><li><span><a href="#What-percentage-of-movies-is-for-adults" data-toc-modified-id="What-percentage-of-movies-is-for-adults-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>What percentage of movies is for adults</a></span></li><li><span><a href="#Statistics-about-duration" data-toc-modified-id="Statistics-about-duration-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Statistics about duration</a></span></li><li><span><a href="#Which-entry-is-this-damn-longest-movie?" data-toc-modified-id="Which-entry-is-this-damn-longest-movie?-3.2.4"><span class="toc-item-num">3.2.4&nbsp;&nbsp;</span>Which entry is this damn longest movie?</a></span></li></ul></li></ul></li></ul></div>

Personal project to practice with SQL request.

Using 2 tsv files (the titles and the ratings) uploaded in a Database and then using various queries to gather details

Information courtesy of
IMDb
(http://www.imdb.com).
Used with permission.

In [1]:
import psycopg2

from credentials import *

Starting section 2, I realize I could improve the code by factoring the recurrent connection and closing to database, with the following class:

In [13]:
class MyDB(object):

    def __init__(self):
        self._db_connection = psycopg2.connect(
   database="postgres", user=POSTGRESUSER, password=POSTGRESPWD, host='127.0.0.1', port= '5432'
)
        self.cursor = self._db_connection.cursor()

    def __enter__(self):
        return self
    
    def __del__(self):
        self._db_connection.close()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.cursor.close()
        self._db_connection.close()


# Creation of Database

In [5]:
#establishing the connection
conn = psycopg2.connect(
   database="postgres", user=POSTGRESUSER, password=POSTGRESPWD, host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Preparing query to create a database
sql = '''CREATE database imdb''';

#Creating a database
cursor.execute(sql)
print("Database created successfully........")

#Closing the connection
conn.close()

ActiveSqlTransaction: CREATE DATABASE cannot run inside a transaction block


# Creations of the tables

## Table structure

Checking the structure from the tsv file, description also available from website: https://www.imdb.com/interfaces/

In [3]:
!head -3 data/title.basics.tsv

tconst	titleType	primaryTitle	originalTitle	isAdult	startYear	endYear	runtimeMinutes	genres
tt0000001	short	Carmencita	Carmencita	0	1894	\N	1	Documentary,Short
tt0000002	short	Le clown et ses chiens	Le clown et ses chiens	0	1892	\N	5	Animation,Short


In [45]:
!head -3 data/title.ratings.tsv

tconst	averageRating	numVotes
tt0000001	5.7	1690
tt0000002	6.0	210


## Table basics

In [6]:
#establishing the connection
conn = psycopg2.connect(
   database="postgres", user=POSTGRESUSER, password=POSTGRESPWD, host='127.0.0.1', port= '5432'
)


#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Preparing query to create a database
sql = '''CREATE TABLE basics(
    tconst CHAR (9) PRIMARY KEY,
    titleType TEXT,
    primaryTitle TEXT,
    originalTitle TEXT,
    isAdult BOOLEAN,
    startYear SMALLINT,
    endYear SMALLINT,
    runtimeMinutes SMALLINT,
    genres TEXT
    )        
        ''';

#Creating a database
cursor.execute(sql)

conn.commit()
print("Table created successfully........")

#Closing the connection
conn.close()

DuplicateTable: relation "basics" already exists


Checking if table is created in DB and try the class at the same occasion

In [19]:
query = """SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='public'
   AND table_type='BASE TABLE';"""


with MyDB() as my_db:
    cursor = my_db.cursor
    cursor.execute(query)
    results = cursor.fetchall()
    print(results)

[('basics',), ('ratings',)]


Adding a try / except block to catch errors

In [16]:
with MyDB() as my_db:
    try:
        cursor = my_db.cursor
        cursor.execute(sql)
        my_db._db_connection.commit()
    except Exception as err:
        print(f"The following error was raised:\n{err}")

The following error was raised:
name 'sql' is not defined


Fixing type for the column Primary key. Turns out some values have more that 9 characters, swithcing to a generic varchar

In [21]:
fix_column_basics = """ALTER TABLE basics
ALTER COLUMN tconst
TYPE VARCHAR
"""


with MyDB() as my_db:
    try:
        cursor = my_db.cursor
        cursor.execute(fix_column_basics)
        my_db._db_connection.commit()
    except Exception as err:
        print(f"The following error was raised:\n{err}")

Fixing type for the column runtime minutes as one entry is longer than a smallint. Turns out some values have more that 9 characters, swithcing to a generic varchar

In [27]:
fix_column_basics_runtime = """ALTER TABLE basics
ALTER COLUMN runtimeMinutes
TYPE INTEGER
"""


with MyDB() as my_db:
    try:
        cursor = my_db.cursor
        cursor.execute(fix_column_basics_runtime)
        my_db._db_connection.commit()
        print("column type changed")
    except Exception as err:
        print(f"The following error was raised:\n{err}")

column type changed


## table ratings

In [18]:
query_table_ratings = '''CREATE TABLE ratings(
    tconst CHAR (9) PRIMARY KEY,
    averageRating NUMERIC (3,1),
    numVotes INTEGER
    );        
        '''

with MyDB() as my_db:
    try:
        cursor = my_db.cursor
        cursor.execute(query_table_ratings)
        my_db._db_connection.commit()
    except Exception as err:
        print(f"The following error was raised:\n{err}")

Fixing type for the column Primary key. Turns out some values have more that 9 characters, swithcing to a generic varchar

In [22]:
fix_column_basics = """ALTER TABLE ratings
ALTER COLUMN tconst
TYPE VARCHAR
"""


with MyDB() as my_db:
    try:
        cursor = my_db.cursor
        cursor.execute(fix_column_basics)
        my_db._db_connection.commit()
        print("Operation done")
    except Exception as err:
        print(f"The following error was raised:\n{err}")

Operation done


## Inserting the data

In [23]:
with MyDB() as my_db:
    try:
        cursor = my_db.cursor
        with open('data/title.ratings.tsv', 'r') as f:
            next(f) # Skip the header row.
            cursor.copy_from(f, 'ratings', sep='\t')
        my_db._db_connection.commit()
    except Exception as err:
        print(f"The following error was raised:\n{err}")

In [28]:
with MyDB() as my_db:
    try:
        cursor = my_db.cursor
        with open('data/title.basics.tsv', 'r') as f:
            next(f) # Skip the header row.
            cursor.copy_from(f, 'basics', sep='\t')
        my_db._db_connection.commit()
        print("Data imported")
    except Exception as err:
        print(f"The following error was raised:\n{err}")

Data imported


# Playing with queries

## Function to ask the queries

In [59]:
def askdb(query,output=1):
    with MyDB() as my_db:
        try:
            cursor = my_db.cursor
            cursor.execute(query)
            if output == 1 :
                #When asking one single value
                result = cursor.fetchone()[0]
            elif output == "row":
                #Asking for several values but on a single row result
                result = cursor.fetchone()
            else:
                #When many lines are expected
                result = [r[0] for r in cursor.fetchall()]
            return result
        except Exception as err:
            print(f"The following error was raised:\n{err}")

Testing default setting with one answer expected

In [38]:
sql = "Select COUNT(*) from ratings"
 
answer = askdb(sql)
print(answer)

1132236


Testing with multiple answers expected

In [44]:
sql = "Select DISTINCT (startYear) from basics"
 
answer = askdb(sql,"many")
print(answer)

[1959, 1907, 2021, 1964, 1969, None, 1889, 1991, 1989, 1974, 1977, 1971, 1935, 1898, 2022, 2017, 1983, 1899, 1891, 2009, 1958, 1973, 2013, 2003, 1933, 1953, 1903, 1892, 1912, 1997, 1905, 1883, 1897, 1894, 2016, 2018, 1874, 1994, 1963, 2014, 1928, 2025, 1924, 1987, 1878, 2000, 1948, 1932, 1985, 1908, 1941, 2115, 1947, 1938, 1925, 1911, 1895, 1906, 1960, 1951, 1915, 1981, 1966, 1916, 1978, 1968, 1944, 2006, 2020, 1962, 1926, 1923, 2012, 1954, 2024, 1988, 1896, 2008, 1945, 1943, 1956, 1940, 1890, 2028, 1984, 1934, 2005, 1913, 1885, 1914, 1904, 2015, 1990, 1930, 1993, 2002, 1979, 2026, 2004, 1980, 1919, 1922, 1975, 1986, 1970, 1921, 1936, 1982, 1901, 1942, 1995, 1992, 2023, 1920, 2010, 1887, 1939, 2019, 1996, 1931, 1967, 2027, 2011, 1965, 1909, 1949, 1881, 1888, 1918, 1946, 1917, 1998, 1976, 1972, 2007, 1929, 1927, 1902, 1961, 1952, 2001, 1893, 1910, 1999, 1950, 1955, 1957, 1937, 1900]


## Simple queries

### How many different genre combinations are there

In [50]:
sql = "Select DISTINCT (genres) from basics"
 
answer = askdb(sql,"many")
print(f"There is a total of {len(answer)} combinations of genres indicated. Here is the complete list:")
print(answer)

There is a total of 2276 types of genres indicated. Here is the complete list:
[None, 'Comedy,Drama,News', 'Crime,Talk-Show', 'Action,Adult,Thriller', 'Action,Crime,Sport', 'Mystery,Romance,Thriller', 'Drama,Sci-Fi,Thriller', 'Comedy,Crime,Film-Noir', 'News,Reality-TV,Talk-Show', 'Action,Adult,Sci-Fi', 'Action,Biography,War', 'Fantasy,Musical,Thriller', 'Fantasy,History,News', 'Animation,Comedy,Thriller', 'Animation,Drama,News', 'Documentary,Music,Musical', 'Biography,Music,News', 'Action,Fantasy,Talk-Show', 'Documentary,Family,Mystery', 'Adult,Romance,Sport', 'Documentary,Fantasy,Mystery', 'Animation,Comedy,Horror', 'Animation,Family,Musical', 'Animation,Biography,Comedy', 'Crime,Mystery,Reality-TV', 'Horror,Romance,Western', 'Animation,Comedy,News', 'Fantasy,Mystery,Thriller', 'Biography,Comedy,War', 'Game-Show,Musical,Romance', 'History,Music', 'Drama,Reality-TV,Thriller', 'Biography,Crime,Western', 'Family,News,Sport', 'Documentary,News', 'Family,Musical,Western', 'Fantasy,Music,Mu

### What percentage of movies is for adults

In [51]:
sql = "Select ROUND(COUNT(isAdult)/COUNT(*),2) from basics"
answer = askdb(sql)
print(f"The percentage of adult movies in the database is {answer}%")

The percentage of adult movies in the database is 1.00%


### Statistics about duration

In [63]:
sql = "SELECT ROUND(AVG(runtimeMinutes),2), PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY runtimeMinutes), MIN(runtimeMinutes),MAX(runtimeMinutes)  from basics"
 
answer = askdb(sql,"row")
print(f"The average length of a movie is {answer[0]}, while the median observed is {answer[1]}. \nThe movies duration in the dataset vary from {answer[2]} minutes to {answer[3]} minutes.")

The average length of a movie is 44.05, while the median observed is 30. 
The movies duration in the dataset vary from 0 minutes to 51420 minutes.


### Which entry is this damn longest movie?

In [64]:
sql = "SELECT primaryTitle,startYear from basics WHERE runtimeMinutes = 51420"
answer = askdb(sql,"row")
print(f"The longest movie's title is {answer[0]} and was released in {answer[1]}")

('tt8273150', 'movie', 'Logistics', 'Logistics', False, 2012, None, 51420, 'Documentary')


Side note, IMDB confirm in [the movie's trivia page](https://www.imdb.com/title/tt8273150/trivia?ref_=tt_trv_trv) that it is indeed the longest ever!